# Inference Demo: Human-Centered HIV Drug Resistance Interpretation

This notebook demonstrates a minimal, reproducible example of how an HIV drug resistance
machine-learning model can be combined with a HAI-DEF model (Gemma / MedGemma family)
for human-centered explanation.

This is a **decision-support demonstration only**.
No diagnosis or treatment recommendations are provided.

In [ ]:
pip_install = __import__("subprocess").run(
    ["pip", "install", "-q", "huggingface_hub"],
    capture_output=True
)
print("huggingface_hub installed")

huggingface_hub installed


In [ ]:
from huggingface_hub import login

login()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "google/medgemma-1.5-4b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("MedGemma loaded successfully")

`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

MedGemma loaded successfully


In [ ]:
case_json = """
{
  "case_id": "Case_2",
  "case_type": "Multiple mutations",
  "gene": "HIV protease",
  "mutations": ["M46I", "I54V", "V82A", "I84V", "L90M"],
  "model_prediction": {
    "resistance_probability": 0.938,
    "predicted_label": "Resistant"
  },
  "confidence": "High",
  "limitations": [
    "Model trained mainly on subtype B data",
    "Treatment history not included"
  ],
  "disclaimer": "Decision support only, not a diagnosis"
}
"""


Clinician explanation (generated)

In [ ]:
prompt_clinician = f"""
You are a clinical decision-support assistant.

The following JSON comes from a machine learning model predicting HIV drug resistance.
This output is NOT a diagnosis.

JSON:
{case_json}

Tasks:
1. Explain the resistance risk in clinician-friendly language
2. Identify which mutations most strongly drive the prediction
3. State confidence and limitations clearly
4. Mention when caution is required

Do NOT recommend specific drugs.
"""


In [ ]:
inputs = tokenizer(prompt_clinician, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=250,
    do_sample=False
)

clinician_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(clinician_response)


Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.



You are a clinical decision-support assistant.

The following JSON comes from a machine learning model predicting HIV drug resistance.
This output is NOT a diagnosis.

JSON:

{
  "case_id": "Case_2",
  "case_type": "Multiple mutations",
  "gene": "HIV protease",
  "mutations": ["M46I", "I54V", "V82A", "I84V", "L90M"],
  "model_prediction": {
    "resistance_probability": 0.938,
    "predicted_label": "Resistant"
  },
  "confidence": "High",
  "limitations": [
    "Model trained mainly on subtype B data",
    "Treatment history not included"
  ],
  "disclaimer": "Decision support only, not a diagnosis"
}


Tasks:
1. Explain the resistance risk in clinician-friendly language
2. Identify which mutations most strongly drive the prediction
3. State confidence and limitations clearly
4. Mention when caution is required

Do NOT recommend specific drugs.



In [ ]:
PATIENT_EXPLANATION_FINAL = """
The result suggests that some HIV medicines may not work as well against the virus in this situation.
This information comes from a computer model that analyzes patterns in the virus sample,
rather than information about your overall health.

These predictions are not certain and are used as one piece of information.
Healthcare professionals review results like this together with other information
to better understand what they may mean.
"""

print(PATIENT_EXPLANATION_FINAL)


The result suggests that some HIV medicines may not work as well against the virus in this situation.
This information comes from a computer model that analyzes patterns in the virus sample,
rather than information about your overall health.

These predictions are not certain and are used as one piece of information.
Healthcare professionals review results like this together with other information
to better understand what they may mean.



In [ ]:
inputs = tokenizer(prompt_patient, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=120,
    do_sample=False
)

patient_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(patient_response)


Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.



You are explaining health information to a patient.

Context:
This information describes how well some HIV medicines may work against the virus.
It does NOT describe the patient’s genes, health status, prognosis, or treatment plan.

Facts you may use:
- A computer model analyzed patterns in the virus
- Some medicines may work less effectively
- Predictions are uncertain
- This is only one piece of information

Rules (MANDATORY):
- Use calm, reassuring language
- Do NOT mention genes, mutations, viral load, or codes
- Do NOT give advice or suggestions of any kind
- Do NOT suggest talking about changing medicines or options
- Do NOT imply worsening health or outcomes
- Avoid fear-inducing language
- Under 120 words

Task:
Explain what this result means for the patient.

The result suggests that some HIV medicines may not work as well against the virus in this situation.
This information comes from a computer model that looks at patterns in the virus, not from your body or overall health

## Notes on Safety and Intended Use

- This demonstration is **decision-support only**
- No diagnosis or treatment advice is provided
- Patient-facing explanations are fixed to ensure safe communication
- This notebook is intended as a minimal reproducible example for the MedGemma Impact Challenge

In [45]:
rm -rf hiv-medgemma-impact

In [46]:
!git clone https://github.com/SandhyaSingh-research/hiv-medgemma-impact.git
%cd hiv-medgemma-impact

Cloning into 'hiv-medgemma-impact'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 34 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (34/34), 27.80 KiB | 13.90 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/hiv-medgemma-impact/hiv-medgemma-impact


In [47]:
cp /content/drive/MyDrive/medgemma_competition/inference_demo.ipynb

cp: missing destination file operand after '/content/drive/MyDrive/medgemma_competition/inference_demo.ipynb'
Try 'cp --help' for more information.


In [48]:
pwd

'/content/hiv-medgemma-impact/hiv-medgemma-impact'

In [49]:
cp /content/drive/MyDrive/medgemma_competition/inference_demo.ipynb .

In [50]:
cp /content/drive/MyDrive/medgemma_competition/inference_demo.ipynb .

In [51]:
!git add .
!git commit -m "Add clean inference demo notebook and docs"
!git push

[main eedb374] Add clean inference demo notebook and docs
 1 file changed, 1 insertion(+)
 create mode 100644 inference_demo.ipynb
fatal: could not read Username for 'https://github.com': No such device or address


In [40]:
# Configure your Git user name and email
# Replace 'Your Name' and 'you@example.com' with your actual Git credentials
!git config --global user.name "SandhyaSingh-research"
!git config --global user.email "sandhya258singh@gmail.com"

print("Git user name and email configured.")

Git user name and email configured.


In [43]:
!git add
!git commit -m "Add clean inference demo notebook and docs"
!git push

Nothing specified, nothing added.
hint: Maybe you wanted to say 'git add .'?
hint: Turn this message off by running
hint: "git config advice.addEmptyPathspec false"
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address
